In [2]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import re

In [3]:
genename = []                                                       # Creamos una lista vacia llamada genename
for i in range(11):                                                 # Para i en una rango de (0 a 10)
    if i == 0:                                                      # Si i es igual a 0   
        data = pd.read_excel("Genes identificados con vecinos.xlsx",# Nombre de la data
                    sheet_name=i,                                   # Numero de hoja, en este caso sera 0
                     index_col="N°",                                # La columna que sera el index
                    skiprows=4)                                     # saltar las 4 primeras filas
    else:
        data = pd.read_excel("Genes identificados con vecinos.xlsx",# Nombre de la data
                        sheet_name=i,                               # Numero de la hoja
                         index_col="N°",                            # La columna que sera el index
                        skiprows=1)                                 # saltar la primera fila
    data.to_csv("genes.csv")                                        # Convertimos la data en un csv
    
    ef = pd.read_csv('genes.csv')              # leemos la data y lo asignamos a la variable ef
    genename.extend(list(ef["Gen Abrev"]))     # Agregamos la columna Gen Abrev a la lista genename
                                                                         
entry = []                # Creamos una lista varia llamada entry 
entry_name=[]             # Creamos una lista varia llamada entry_name 
protein =[]               # Creamos una lista varia llamada protein
gene =[]                  # Creamos una lista varia llamada gene
organism =[]              # Creamos una lista varia llamada organism 
status =[]                # Creamos una lista varia llamada status
Biologicalprocess = []    # Creamos una lista varia llamada Biologicalprocess 
Molecular_funtion=[]      # Creamos una lista varia llamada Molecular_funtion
Biological_process=[]     # Creamos una lista varia llamada Biological_process
Ligand=[]                 # Creamos una lista varia llamada Ligand

for i in genename:                                                                   # para i cada elemento de genename
    url = "https://www.uniprot.org/uniprot/?query="+str(i)+"+bos+taurus&sort=score"  # Concatenamos cadenas para las paginas web
    html = requests.get(url)                                # hacemos una peticion para obtener el contenido de la web
    soup = BeautifulSoup(html.content, 'html.parser')       # Extraemos la informacion en formato html
    etiquetas = soup.find('tr',class_="entry selected-row") # de lo extraido vamo a encontrar la etiqueta con nombre 'tr' y clase "entry selected-row"
    if etiquetas == None:                                   # Si etiquetas es vacio
        entry.append("na")                                  # La lista entry le agregamos un na
        entry_name.append("na")                             # La lista entry_name le agregamos un na
        protein.append("na")                                # La lista Protein le agregamos un na
        gene.append("na")                                   # La lista gene le agregamos un na
        organism.append("na")                               # La lista organism le agregamos un na
        status.append("na")                                 # La lista status le agregamos un na
        Biologicalprocess.append("na")                      # La lista Biologicalprocess le agregamos un na
        Molecular_funtion.append("na")                      # La lista Molecular_funtion le agregamos un na
        Biological_process.append("na")                     # La lista Biological_process le agregamos un na
        Ligand.append("na")                                 # La lista Ligand le agregamos un na
    else:                                                   # Si etiquetas no es vacio
        a =etiquetas.find("td",class_="entryID").text       # de lo extraido vamos a encontrar la etiqueta con nombre 'td' y clase "entryID" y solo obtendremos el texto y asignamo a la variable a
        entry.append(a)                                     # el string "a" le adjuntamos a la lista entry
        entry_name.append(etiquetas.findAll("td")[2].text)  # de etiquetas vamos a obtener el texto de la segunda etiqueta "td" y lo juntamos a la lista entry_name
        url1 = "https://www.uniprot.org/uniprot/"+str(a)    # Concatenamos para las paginas web y le asignamos a la variable url1
        html1 = requests.get(url1)                          # hacemos una peticion para obtener el contenido de la web
        soup1 = BeautifulSoup(html1.content, 'html.parser') # Extraemos la informacion en formato html y le asignamos a la variable soup1

        etiquetas1 = soup1.find("div", id="entry-overview") # de lo extraido de soup1 vamos a encontrar la etiqueta con nombre 'div' y id "entry-overview" y asignamo a la variable "etiquetas1" 

        protein.append(etiquetas1.find("h1", property="name").text)            # De lo extraido en etiquetas1 vamos a encontrar las etiquetas con nombre "h1" y property "name" y le agregamos a protein el texto 
        gene.append(etiquetas1.find("div", id="content-gene").text)            # De lo extraido en etiquetas1 vamos a encontrar las etiquetas con nombre "div" y id "content-gene" y le agregamos a gene el texto
        organism.append(etiquetas1.find("div", id="content-organism").text)    # De lo extraido en etiquetas1 vamos a encontrar las etiquetas con nombre "div" y id "content-organism" y le agregamos a organism el texto
        status.append(etiquetas1.find("a", href="/manual/entry_status").text)  # De lo extraido en etiquetas1 vamos a encontrar las etiquetas con nombre "a" y href debe de contener "/manual/entry_status" y le agregamos a status el texto
        
        function = soup1.find("div", id="function")                                           # De lo extraido en soup vamos a encontrar las etiquetas "div" y div "function" para asignarle a una variable function
        
        if function != None:                                                                  # Si function no es vacio
            etiquetas2 = function.find("ul", class_="noNumbering biological_process")         # De lo extraido en function vamos a buscar las etiquetas "ul" y la clase "noNumbering biological_process" asignandole a la variable etiquetas2

            if etiquetas2 != None:                                                            # Si etiquetas2 no es vacio
                biological = etiquetas2.findAll("a",href=re.compile("https://www.ebi.ac.uk")) # De etiquetas dos vamos a encontrar todos las etiquetas "a" y los href que tengan "https://www.ebi.ac.uk"

                h=""                           # Creamos una string vacia con nombre h
                for i in biological:           # i tomara cada elemento de biological
                    h+= i.text+". "            # Concatenamos cadenas 
                Biologicalprocess.append(h)    # Recien agregamos h a la lista Biologicalprocess
            else:                              # Caso contrario
                Biologicalprocess.append("na") # Agregamos "na

            etiquetas3 = function.find("table", class_="databaseTable") # De function vamos a encontrar la etiqueta "table" y la clase "databaseTable" y le asignamos a etiquetas3
            if etiquetas3 != None:                                      # Si etiquetas3 no es vacio
                keywords = etiquetas3.findAll("tr")                     # De etiquetas3 vamos a encontrar todos los tr y le asignamos a keywords
                n=0  
                lista1=""                                               # Creamos string vacia
                lista2=""                                               # Creamos string vacia
                lista3 =""                                              # Creamos string vacia
                for i in keywords:                                      # i tomara cada elemento de keywords
                    a = i.findAll("a", href=re.compile("keywords"))     # en i buscamos todas las etiquetas que tengan "a" y dentro del href "keywords" y lo asignamos a "a"
                    if i.find("td").text == "Molecular function":       # si en i buscamos el texto de las etiquetas "td" y si es igual a "Molecular function"
                        for i in a :                                    # i tomara cada elemento de a
                            lista1+=i.text+". "                         # concatenando cadenas

                    elif i.find("td").text == "Biological process":     # Caso contrario, si en i buscamos el texto de las etiquetas "td" y si es igual a "Biological process"                       
                        for i in a :                                    # i tomara cada elemento de a
                            lista2+= i.text+". "                        # concatenando cadenas
                         

                    elif i.find("td").text == "Ligand":                 # Caso contrario, si en i buscamos el texto de las etiquetas "td" y si es igual a "Ligand"
                        for i in a :                                    # i tomara cada elemento de a
                            lista3+= i.text+". "                        # concatenamos cadenas
                          

                if len(lista1)!=0:                   # Si la longitud de la lista1 es diferente de 0
                    Molecular_funtion.append(lista1) # Agregamos lista1 a la lista "Molecular_funtion"
                else:                                # Caso contrario
                    Molecular_funtion.append("na")   # Le agregamos "na"

                if len(lista2)!=0:                    # Si la longitud de la lista2 es diferente de 0
                    Biological_process.append(lista2) # Agregamos lista2 a la lista "Biological_process"
                else:                                 # Caso contrario 
                    Biological_process.append("na")   # Le agregamos "na"

                if len(lista3) !=0:       # Si la longitud de la lista3 es diferente de 0
                    Ligand.append(lista3) # Agregamos lista3 a la lista "Ligand"
                else:                     # Caso contrario
                    Ligand.append("na")   # Le agregamos "na" 

            else:                               # Caso Contrario
                Molecular_funtion.append("na")  # Agregamos "na" a la lista Molecular_funtion
                Biological_process.append("na") # Agregamos "na" a la lista Biological_process
                Ligand.append("na")             # Agregamos "na" a la lista Ligand
        else:                               # Caso contario
            Biologicalprocess.append("na")  # Agregamos "na" a la lista Biologicalprocess
            Molecular_funtion.append("na")  # Agregamos "na" a la lista Molecular_funtion
            Biological_process.append("na") # Agregamos "na" a la lista Biological_process
            Ligand.append("na")             # Agregamos "na" a la lista Ligand


# Imprimimos la longitudes para corroborrar que todos tienen la misma longitud para hace un  dataFrame               
print(len(entry))
print(len(entry_name))
print(len(protein))
print(len(gene))
print(len(organism))
print(len(status))
print(len(Biologicalprocess))
print(len(Molecular_funtion))
print(len(Biological_process))
print(len(Ligand))

# Creamos un diccionario con todas las listas creadas y sus llaves el nombre que sera para las columnas
diccionario={"GENEABREV":genename,"ENTRY":entry,"ENTRY NAME":entry_name, "PROTEIN":protein,
            "GENE":gene,"ORGANISM":organism,"STATUS":status, "GO - BIOLOGICAL PROCESS":Biologicalprocess,
            "MOLECULAR FUNCION":Molecular_funtion, "BIOLOGICAL PROCESS":Biological_process,
            "LIGARD":Ligand}
# Creammos un DataFrame con el diccionario
df = pd.DataFrame(diccionario, columns=["GENEABREV","ENTRY","ENTRY NAME","PROTEIN", "GENE","ORGANISM","STATUS",
                                       "GO - BIOLOGICAL PROCESS","MOLECULAR FUNCION","BIOLOGICAL PROCESS","LIGARD"])
# Convertimos el DataFrame a un "csv" 
df.to_csv('TRABAJOGENES.csv') 

# Leemos el "csv" y le asignamos a la variable trabajo
trabajo = pd.read_csv('TRABAJOGENES.csv', skiprows=1,          
                 names=["GENEABREV","ENTRY","ENTRY NAME","PROTEIN", "GENE","ORGANISM","STATUS",
                                       "GO - BIOLOGICAL PROCESS","MOLECULAR FUNCION","BIOLOGICAL PROCESS","LIGARD"] )
trabajo # Mostramos
     

KeyboardInterrupt: 